# CO2 analysis

This notebook is being used to prepare data and figures for the upcoming APERC Annual CO2 Emissions Report, due for publication in June 2020. The data and results here are strictly preliminary at this point.

A final interactive notebook will be made available at publication.

To use this notebook:
1. import the libraries below
2. import the prepared data
3. Plot all economies together
4. Plot each economy separately. You can select either fuels or sectors.

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

import param
import hvplot.pandas
import holoviews  as hv

from bokeh.palettes import brewer
from bokeh.palettes import Category20
from bokeh.palettes import viridis
from bokeh.palettes import inferno
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.models import Legend
from bokeh.plotting import figure

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot

import panel as pn
pn.extension()
import panel.widgets as pnw
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas
output_notebook()

### Read in prepared data:

In [ ]:
df = pd.read_csv('../data/CO2 emissions prepared.csv')

In [ ]:
df = df.sort_values(by='Economy')
economies = df['Economy'].unique().tolist()
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
fuels= ['1. CO2 Emission by Energy',
        '1.1 Coal & Coal Products',
        '1.2 Crude Oil & Petroleum Products',
        '1.3 Gas']

sectors= ['2. CO2 Emission by Sector',
          '2.1 Transformation Sector',
          '2.1.1 Main Activity Producer',
          '2.1.2 Autoproducers',
          '2.1.3 Gas Processing',
          '2.1.4 Loss & Own Use',
          '2.2 Final Energy Consumption Sector',
          '2.2.1 Industry Sector',
          '2.2.2 Transport Sector',
          '2.2.3 Residential & Commercial',
          '2.2.4 Other'
          ]

Two classes:

In [ ]:
class carbon(param.Parameterized):
    fuel_selector = param.ObjectSelector(default="1.1 Coal & Coal Products", objects=fuels)
    year_range = param.Range(default=(1990, 2017), bounds=(1990, 2017))
    show_legend = param.Boolean(True, doc="Show the legend")
        
    title = 'CO2 emissions'
    xlabel = 'Years'
    ylabel = 'Million tons'
   
    def get_data(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        mask = (df['Item'] == self.fuel_selector) & (df['Year'] > start_date) & (df['Year'] <= end_date)
        
        _df = df.loc[mask]
        
        df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
       
    def hv_plot_bar(self):
        data  = self.get_data()
        title = 'Bar chart - by fuel'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.bar(x ='Year', y=names, title=title,width=400, height=500, legend=False, alpha=0.4, stacked=True, rot=90)
        #plot  = plot.options(legend_position='top')
        return plot
    
    def hv_plot_area(self):
        data  = self.get_data()
        title = 'Area chart - by fuel'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.area(x ='Year', y=names, title=title,width=500, height=500, legend=self.show_legend,alpha=0.4)
        #plot  = plot.options(legend_position='bottom_right')
        return plot

c = carbon(name='')   
    
class carbon_sector(param.Parameterized):
    sector_selector = param.ObjectSelector(default="2.1 Transformation Sector", objects=sectors)
    year_range = param.Range(default=(1990, 2017), bounds=(1990, 2017))
    show_legend = param.Boolean(True, doc="Show the legend")
    
    def get_data(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        mask = (df['Item'] == self.sector_selector) & (df['Year'] > start_date) & (df['Year'] <= end_date)
        
        _df = df.loc[mask]
        
        df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
    
    def hv_plot_bar(self):
        data  = self.get_data()
        title = 'Bar chart - by sector'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.bar(x ='Year', y=names, title=title,width=400, height=500, legend=False, alpha=0.4, stacked=True, rot=90,
                                xlim=self.year_range)
        return plot
        
    def hv_plot_area(self):
        data = self.get_data()
        title = 'Area chart - by sector'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.area(x ='Year', y=names, title=title,width=500, height=500, legend=self.show_legend,alpha=0.4)
        return plot

cs = carbon_sector(name='') 

In [ ]:
pn.Column(pn.Row(c.param,c.hv_plot_area, c.hv_plot_bar),
          pn.Row(cs.param,cs.hv_plot_area, cs.hv_plot_bar))

One class:

In [ ]:
class carbon(param.Parameterized):
    fuel_selector = param.ObjectSelector(default="1.1 Coal & Coal Products", objects=fuels)
    sector_selector = param.ObjectSelector(default="2.1 Transformation Sector", objects=sectors)
    year_range = param.Range(default=(1990, 2017), bounds=(1990, 2017))
    show_legend = param.Boolean(True, doc="Show the legend")
        
    title = 'CO2 emissions'
    xlabel = 'Years'
    ylabel = 'Million tons'
   
    def get_data_fuel(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        mask = (df['Item'] == self.fuel_selector) & (df['Year'] > start_date) & (df['Year'] <= end_date)
        
        _df = df.loc[mask]
        
        df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
       
    def hv_plot_bar_fuel(self):
        data  = self.get_data_fuel()
        title = 'Bar chart - by fuel'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.bar(x ='Year', y=names, title=title,width=400, height=500, legend=False, alpha=0.4, stacked=True, rot=90)
        #plot  = plot.options(legend_position='top')
        return plot
    
    def hv_plot_area_fuel(self):
        data  = self.get_data_fuel()
        title = 'Area chart - by fuel'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.area(x ='Year', y=names, title=title,width=500, height=500, legend=self.show_legend,alpha=0.4)
        #plot  = plot.options(legend_position='bottom_right')
        return plot
    
    def get_data_sector(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        mask = (df['Item'] == self.sector_selector) & (df['Year'] > start_date) & (df['Year'] <= end_date)
        
        _df = df.loc[mask]
        
        df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
    
    def hv_plot_bar_sector(self):
        data  = self.get_data_sector()
        title = 'Bar chart - by sector'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.bar(x ='Year', y=names, title=title,width=400, height=500, legend=False, alpha=0.4, stacked=True, rot=90,
                                xlim=self.year_range)
        return plot
        
    def hv_plot_area_sector(self):
        data = self.get_data_sector()
        title = 'Area chart - by sector'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.area(x ='Year', y=names, title=title,width=500, height=500, legend=self.show_legend,alpha=0.4)
        return plot

c = carbon(name='')

In [ ]:
pn.Column(c.param,
          pn.Row(c.hv_plot_bar_fuel, c.hv_plot_bar_sector))